In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField,StringType,IntegerType

In [2]:
#Creating Spark Session (It will executed in the Local Not in the Master Node)
spark=SparkSession.builder.master("local[*]").appName("demo").getOrCreate()
spark

23/04/04 17:21:06 WARN Utils: Your hostname, ZSCHN01LP0253L resolves to a loopback address: 127.0.1.1; using 192.168.1.5 instead (on interface wlp0s20f3)
23/04/04 17:21:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/04/04 17:21:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/04/04 17:21:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
empDf = spark.read.option("header",True).option("inferSchema",True).csv("../../Spark-main/employees.csv")
empDf.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [5]:
deptDf = spark.read.option("header",True).option("inferSchema",True).csv("../../Spark-main/departments.csv")
deptDf.show(5)

+-------------+---------------+----------+-----------+
|DEPARTMENT_ID|DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+---------------+----------+-----------+
|           10| Administration|       200|       1700|
|           20|      Marketing|       201|       1800|
|           30|     Purchasing|       114|       1700|
|           40|Human Resources|       203|       2400|
|           50|       Shipping|       121|       1500|
+-------------+---------------+----------+-----------+
only showing top 5 rows



### UDF

In [6]:
from pyspark.sql.functions import udf,col

def upperCase(in_str):
    out_str = in_str.upper()
    return out_str

print(upperCase("hello"))
upperCaseUDF = udf(lambda z : upperCase(z) , StringType())
empDf.select(col("EMPLOYEE_ID") , col("FIRST_NAME"), col("LAST_NAME"), upperCaseUDF(col("FIRST_NAME")), upperCaseUDF(col("LAST_NAME"))).show()

HELLO
+-----------+----------+---------+--------------------+-------------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|<lambda>(FIRST_NAME)|<lambda>(LAST_NAME)|
+-----------+----------+---------+--------------------+-------------------+
|        198|    Donald| OConnell|              DONALD|           OCONNELL|
|        199|   Douglas|    Grant|             DOUGLAS|              GRANT|
|        200|  Jennifer|   Whalen|            JENNIFER|             WHALEN|
|        201|   Michael|Hartstein|             MICHAEL|          HARTSTEIN|
|        202|       Pat|      Fay|                 PAT|                FAY|
|        203|     Susan|   Mavris|               SUSAN|             MAVRIS|
|        204|   Hermann|     Baer|             HERMANN|               BAER|
|        205|   Shelley|  Higgins|             SHELLEY|            HIGGINS|
|        206|   William|    Gietz|             WILLIAM|              GIETZ|
|        100|    Steven|     King|              STEVEN|               KING|
|     

In [7]:
@udf(returnType=StringType())
def upperCaseNew(in_str):
    out_str = in_str.upper()
    return out_str

In [8]:
empDf.select(col("EMPLOYEE_ID") , col("FIRST_NAME"), col("LAST_NAME"), upperCaseNew(col("FIRST_NAME")), upperCaseNew(col("LAST_NAME"))).show()

+-----------+----------+---------+------------------------+-----------------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|upperCaseNew(FIRST_NAME)|upperCaseNew(LAST_NAME)|
+-----------+----------+---------+------------------------+-----------------------+
|        198|    Donald| OConnell|                  DONALD|               OCONNELL|
|        199|   Douglas|    Grant|                 DOUGLAS|                  GRANT|
|        200|  Jennifer|   Whalen|                JENNIFER|                 WHALEN|
|        201|   Michael|Hartstein|                 MICHAEL|              HARTSTEIN|
|        202|       Pat|      Fay|                     PAT|                    FAY|
|        203|     Susan|   Mavris|                   SUSAN|                 MAVRIS|
|        204|   Hermann|     Baer|                 HERMANN|                   BAER|
|        205|   Shelley|  Higgins|                 SHELLEY|                HIGGINS|
|        206|   William|    Gietz|                 WILLIAM|                 